In [9]:
TIME_DATA_JSON = 'time_data_en.json'

In [10]:
# 读取配置
import json

def read_json_file(file_name: str):
    with open(file_name, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

def write_dict_to_json(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

coze_config = read_json_file('config-coze.json')
print(coze_config)

{'space_id': '7295668070847102994', 'bot_id': '7348351797779759111', 'api_key': 'u7yf5zGfvyQiYoIR5kdxNi4C55s2ov4BFQUgAax7Uhtmz8vcAkrMwL6jbtIeCpGq'}


In [11]:
import requests
import json

def post_to_chat_API(query):
    headers = {
        'Authorization': f'Bearer {coze_config["api_key"]}',
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'Host': 'api.coze.com',
        'Connection': 'keep-alive'
    }

    data = {
        "bot_id": f"{coze_config['bot_id']}",
        "user": "1145141919810",
        "query": query,
        "stream": False
    }

    response = requests.post('https://api.coze.com/open_api/v2/chat', headers=headers, data=json.dumps(data))
    
    return response

In [12]:
import time

res_dict = read_json_file(TIME_DATA_JSON)
for i in range(0,3):   # retry
    for hour in range(0,24):
        for minute in range(0,60):
            if minute < 10: 
                minute = "0" + str(minute)
            time_str = f"{hour}:{minute}"
            if time_str in res_dict.keys():
                continue

            print(time_str)
            try:
                LLM_response = json.loads(post_to_chat_API(time_str).content)
                res_dict[time_str] = LLM_response['messages'][0]['content']
                print(res_dict[time_str])
                write_dict_to_json(TIME_DATA_JSON, res_dict)
            except Exception as e:
                print(e)
            time.sleep(5)


0:48
Minutes to one, night's quiet echo,
Moon's glow, silence's perfect duo.

Perched on moon's edge, 'fore morning bell,
Night whispers secrets, in dark it fell.

At midnight's hold, nearly morn's appeal,
Stars' symphony in the quiet we feel.
0:49
Before dawn's break at forty-nine past,
Hushed whispers float in the night, vast.

In quiet night at forty-nine past,
Stars blush softly, secrets amassed.

Forty-nine past, the moon's high flight,
Illuminates the breath of night.
0:50
At fifty past midnight's embrace,
Whispers linger in starlit space.

Comes midnight, the hour grown deep,
At fifty past, while the city sleeps.

Silent night at moments near one,
Midnight’s child, the day is done.
0:51
At midnight's gentle cusp, so near,
Silence whispers for those to hear.

Under the moon at fifty-one past,
In midnight's canvas, memories cast.

As fifty-one strikes in the night's heart,
The universe in silence starts.
0:52
One minute past midnight's call,
In moon's light, shadows softly fall.

